In [65]:
import numpy as np
import pandas as pd
import os

# Camera in PMT1 Spectrometer Module
Import the cluster dataframe for the field of view of the camera (PMT1 Spectrometer Module) and create a new dataframe with the clusters that are actual bioluminescent events.

In [6]:
# cluster dataframe for PMT001 camera, field of view only

# database of the camera you want to analyse the pictures of
file = 'cluster_lucifer_db/TUMPMTSPECTROMETER001_20210401T000000_20220901T000000_image_cluster_merge_npixel15.gz'  

if not os.path.exists(file):
    print("Doesn't exist")
else:
    df1 = pd.read_pickle(file)


# delete background clusters with label 0
df1 = df1[df1.label != 0]


# mask events (20 < n_pixel < 1e5)

limit = 20  # limit from plot
limit_max = 1e5  # limit from plot
parameter = 'n_pixel'

size1 = df1[parameter]  > limit
size1 &= df1[parameter]  < limit_max

df1 = df1[size1]

In [7]:
# get rid of pictures with LEDs on

# import lucifer settings
led = pd.read_pickle('cluster_lucifer_db/lucifer_settings_20210225T152529_20220907T212744.gz')
led = led[~led.t_previous_image.isnull()]
led.t_previous_image = pd.to_datetime(led.t_previous_image, utc=True)

# get the time stamps of all pictures with LEDs in the module above the camera on
t_lidar1 = [i for i in led[led.deviceCode == 'TUMLIDAR001'].t_previous_image]

# df without LED pictures
events1 = df1[~df1['time'].isin(t_lidar1)]

In [61]:
# save df 
events1.to_pickle('TUMPMTSPECTROMETER001_20210401T000000_20220901T000000_events.gz')

-------------------------------------------------------------------------------------------------------------

# Camera in PMT2 Spectrometer Module
Import the cluster dataframe for the field of view of the camera (PMT2 Spectrometer Module) and create a new dataframe with the clusters that are actual bioluminescent events.

In [62]:
# cluster dataframe for PMT001 camera, field of view only

# database of the camera you want to analyse the pictures of
file2 = 'cluster_lucifer_db/TUMPMTSPECTROMETER002_20210401T000000_20220901T000000_image_cluster_merge_npixel15.gz'  

if not os.path.exists(file2):
    print("Doesn't exist")
else:
    df2 = pd.read_pickle(file2)

# delete background clusters with label 0
df2 = df2[df2.label != 0]


# mask events (20 < n_pixel < 1e5)

size2 = df2[parameter]  > limit
size2 &= df2[parameter]  < limit_max

df2 = df2[size2]

In [63]:
# get rid of pictures with LEDs on

# get the time stamps of all pictures with LEDs in the module above the camera on
t_lidar2 = [i for i in led[led.deviceCode == 'TUMLIDAR002'].t_previous_image]

# df without LED pictures
events2 = df2[~df2['time'].isin(t_lidar2)]


# only consider events since 2021-07 because DAQ did not work properly before
events2 = events2[events2.time >= '2021-07']


# get rid of the rest of the pictures with LEDs on (not stored in lucifer data)

# mask area where LED clusters are
led_cluster = events2['center_of_mass_y'] < 400
led_cluster &= events2['center_of_mass_y'] > 200
led_cluster &= events2['center_of_mass_x'] > 700
led_cluster &= events2['center_of_mass_x'] < 820
led_cluster &= events2['n_pixel'] > 10000

events2 = events2[~events2.time.isin(events2[led_cluster].time)]

In [60]:
# save df
events2.to_pickle('TUMPMTSPECTROMETER002_20210701T000000_20220901T000000_events.gz')